In [1]:
import requests
import sqlite3
import json
# This file is used to create the database and populate it with data from the API

In [3]:
r = requests.get('https://www.fruityvice.com/api/fruit/all')
#r.json()

In [ ]:
# json has nested dictionary so I'm removing the nested dictionary
cleanjson = r.json()
for i in range(len(cleanjson)):
    for keys in cleanjson[i]['nutritions']:
        cleanjson[i][keys] = cleanjson[i]['nutritions'][keys]
    cleanjson[i].pop('nutritions')
cleanjson

In [5]:
# create a database from the raw json data
con = sqlite3.connect('fruits.db')
cur = con.cursor()
sql_create_raw_json_table = '''CREATE TABLE raw_json(
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                raw_text text
                            )'''
cur.execute('DROP TABLE IF EXISTS raw_json')
cur.execute(sql_create_raw_json_table)
con.commit()

In [6]:
# insert the raw json data into the database
list_of_strings = []
for row in r.json():
    list_of_strings.append(json.dumps(row))

list_of_tuples = []
for row in list_of_strings:
    list_of_tuples.append((row,))
cur.executemany("INSERT INTO raw_json (raw_text) VALUES (?)",list_of_tuples)
con.commit()
con.close()

In [9]:
# gets raw data from database
def get_raw_data():
    con = sqlite3.connect('fruits.db')
    cur = con.cursor()
    list_of_rows = []
    for row in cur.execute('SELECT * from raw_json'):
        list_of_rows.append(row)
    con.close()
    return list_of_rows

In [12]:
raw_data = get_raw_data()
raw_data

[(1,
  '{"genus": "Malus", "name": "Apple", "id": 6, "family": "Rosaceae", "order": "Rosales", "nutritions": {"carbohydrates": 11.4, "protein": 0.3, "fat": 0.4, "calories": 52, "sugar": 10.3}}'),
 (2,
  '{"genus": "Prunus", "name": "Apricot", "id": 35, "family": "Rosaceae", "order": "Rosales", "nutritions": {"carbohydrates": 3.9, "protein": 0.5, "fat": 0.1, "calories": 15, "sugar": 3.2}}'),
 (3,
  '{"genus": "Musa", "name": "Banana", "id": 1, "family": "Musaceae", "order": "Zingiberales", "nutritions": {"carbohydrates": 22, "protein": 1, "fat": 0.2, "calories": 96, "sugar": 17.2}}'),
 (4,
  '{"genus": "Rubus", "name": "Blackberry", "id": 64, "family": "Rosaceae", "order": "Rosales", "nutritions": {"carbohydrates": 9, "protein": 1.3, "fat": 0.4, "calories": 40, "sugar": 4.5}}'),
 (5,
  '{"genus": "Fragaria", "name": "Blueberry", "id": 33, "family": "Rosaceae", "order": "Rosales", "nutritions": {"carbohydrates": 5.5, "protein": 0, "fat": 0.4, "calories": 29, "sugar": 5.4}}'),
 (6,
  '{"g

In [18]:
# column names that I want to keep, which includes the name of the fruit and the nutrition info
column_names = ['name','carbohydrates','protein','fat','calories','sugar']


In [22]:
# prunes the raw data to only include the columns I want 
def data_extractor(list_of_tuples):
    list_of_relevant_info = [] # contains the relevant info which is a list of dictionaries

    for x,y in list_of_tuples: # x is id, y is raw_text
        temp_dict = {} # temporary dictionary to store the relevant info
        parsed_json = json.loads(y) # parse the json, is a dictionary

        # The raw json has a nested dictionary so I'm putting the nested info into the main dictionary
        for key in parsed_json['nutritions']:
            parsed_json[key] = parsed_json['nutritions'][key]
    
        for k,v in parsed_json.items(): # if key is in column_names, then add it to the dictionary, which has the relevant info
            if k in column_names:
                temp_dict[k] = v
        
        list_of_relevant_info.append(temp_dict) 
    
    return list_of_relevant_info




In [24]:
cleaned_data = data_extractor(raw_data)

In [25]:
#['name','carbohydrates','protein','fat','calories','sugar']
sql_create_cleaned_table = '''CREATE TABLE clean_data(
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name text,
                                carbohydrates real,
                                protein real,
                                fat real,
                                calories real,
                                sugar real
                                )'''

In [26]:
con = sqlite3.connect('fruits.db')
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS clean_data')
cur.execute(sql_create_cleaned_table)
con.close()

In [27]:
# creates list of tuples containing all the values so I can insert it into the database
def json_extractor(list_of_dicts):
    list_of_tuples = []
    for row in list_of_dicts: 

        temp_tuple = tuple(x for x in row.values())
        list_of_tuples.append(temp_tuple)
    return list_of_tuples


In [ ]:
holder = json_extractor(cleaned_data)
holder

In [29]:
# ['name','carbohydrates','protein','fat','calories','sugar']
# insert the list of tuples into the database
con = sqlite3.connect('fruits.db')
cur = con.cursor()
cur.executemany("INSERT INTO clean_data (name,carbohydrates,protein,fat,calories,sugar) VALUES (?,?,?,?,?,?)",holder)
con.commit()
con.close()
